<a href="https://colab.research.google.com/github/hechen2020/19ma573HuixinChen/blob/master/src/hw_fourier_heston.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Calculate call price for heston model with given parameters

- Heston

 - init_state=[100., .04], r=.05, kappa=1.2, theta=.04, xi=.3, rho=.5
 
- Vanilla Option

 - otype = 1, strike = 100, maturity = 1., market_price = 15.

In [1]:
'''======
run once, then comment it off, and run again
========'''
!git clone https://github.com/hechen2020/19ma573HuixinChen.git

Cloning into '19ma573HuixinChen'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 322 (delta 49), reused 0 (delta 0), pack-reused 176
Receiving objects: 100% (322/322), 1.23 MiB | 1.19 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [2]:
cd ./19ma573HuixinChen/src/

/content/19ma573HuixinChen/src


In [0]:
from contract_v01 import VanillaOption
from sde_1d_v01 import Gbm_1d, bsm_price

In [0]:
import numpy as np
import scipy as s
import math

For Heston Model, we have the Characteristic Function of Log Asset Price as

$$\phi(u)=\frac{e^{iulnS_0+iurt+\frac{\kappa\theta t(\kappa-i\rho\xi u)}{\xi^2}}}{(cosh\frac{\gamma t}{2}+\frac{\kappa-i\rho\xi u}{\gamma}sinh\frac{\gamma t}{2})^{\frac{2\kappa\theta}{\xi^2}}}\times e^{\frac{-(u^2+iu)v_0}{\gamma coth\frac{\gamma t}{2}+\kappa-i\rho\xi u}}$$

where

$$\gamma=\sqrt{\xi^2(u^2+iu)+(\kappa-i\rho\xi u)^2}$$

$S_0$ and $v_0$ are initial price and volatility

The call price can be evaluated using the characteristic functions as

$$C=S_0I_1-Ke^{-rT}I_2$$

where

$$I_1=\frac{1}{2}+\frac{1}{\pi}\int_0^\infty{Re(\frac{e^{-iuln(K)}\phi(u-i)}{iu\phi(-i)})}du$$

and

$$I_2=\frac{1}{2}+\frac{1}{\pi}\int_0^\infty{Re(\frac{e^{-iuln(K)}\phi(u)}{iu})}du$$

In [0]:
gbm1 = Gbm_1d(init_state = 100., drift_ratio = .05, vol_ratio = .04)

kappa = 1.2
theta = .04
xi = .3
rho = .5
otype = 1
strike = 100.
maturity = 1.
market_price = 15.

In [0]:
def char_fcn(u, s0, v0, r, T, kappa, theta, xi, rho):
  #define gamma
  gamma = np.sqrt(xi**2*(u**2+1j*u)+(kappa-1j*rho*xi*u)**2)
  
  #define phi
  p1 = np.exp(1j*u*np.log(s0)+1j*u*r*T+(kappa*theta*T*(kappa-1j*rho*xi*u))/(xi**2))
  p2 = (np.cosh(gamma*T/2)+(kappa-1j*rho*xi*u)/gamma*np.sinh(gamma*T/2))**((2*kappa*theta)/(xi**2))
  p3 = np.exp((-(u**2+1j*u)*v0)/(gamma*(np.cosh(gamma*T/2)/np.sinh(gamma*T/2))+kappa-1j*rho*xi*u))
  phi = (p1/p2)*p3
  
  return phi

In [0]:
def heston_call(self,
               kappa,
               theta,
               xi,
               rho,
               strike,
               maturity):
  s0 = self.init_state
  v0 = self.vol_ratio
  r = self.drift_ratio
  
  K = strike
  T = maturity  
  
  #function inside the integral
  re1 = lambda u: np.real((np.exp(-1j*u*np.log(K))*char_fcn(u-1j, s0, v0, r, T, kappa, theta, xi, rho))/(1j*u*char_fcn(-1j, s0, v0, r, T, kappa, theta, xi, rho)))
  re2 = lambda u: np.real((np.exp(-1j*u*np.log(K))*char_fcn(u, s0, v0, r, T, kappa, theta, xi, rho))/(1j*u))
  
  #evaluate integral
  int1, err = s.integrate.quad(re1, 0, 5135)#overflow begins to appear for greater upper bound
  int2, err = s.integrate.quad(re2, 0, 5135)#but as an approximation this upperbound is close
  
  #evaluate I1 and I2
  I1 = 1/2 + (1/np.pi)*int1
  I2 = 1/2 + (1/np.pi)*int2
  
  #return call price
  return s0*I1 - K*np.exp(-r*T)*I2

Gbm_1d.heston_call = heston_call

In [8]:
call = gbm1.heston_call(kappa, theta, xi, rho, strike, maturity)
print(call)

9.998984543896427


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in cdouble_scalars
  import sys
